In [1]:
import os
import numpy as np
import pandas as pd
import subprocess
import json
from datetime import datetime
from dateutil import rrule

# PySpark imports
import pyspark
import pyspark.sql.functions as psf
import pyspark.sql.types as pst
import pyspark.sql.functions as F
import pyspark.sql.types as pst
from pyspark.sql.functions import countDistinct
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
print('Ready imports')

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

Ready imports


In [11]:
spark = SparkSession.builder.getOrCreate()

print('Started: ', datetime.now().replace(microsecond=0))

Started:  2023-03-30 15:26:27


In [3]:
# sc.stop()
# print('Stoped:', datetime.now().replace(microsecond=0))

In [12]:
# Read PARQUET file into dataframe
df = spark.read.parquet("all_ipv4_queries.parquet")

df.show()


+--------------------+--------------+--------------------+----------+------+-----+
|              domain|  ipv4_address|          query_name|query_type|   asn|  tld|
+--------------------+--------------+--------------------+----------+------+-----+
|    sildenafilxm.com|162.255.118.51|eforward1.registr...|        MX| 22612|.com.|
|   maxim-verdikt.com|  77.88.21.249|      mx.yandex.net.|        MX|208722|.com.|
| dmitrysoshnikov.com| 77.222.41.100|    mx1.spaceweb.ru.|        MX| 44112|.com.|
|        holoteam.com| 64.233.184.26|alt1.aspmx.l.goog...|        MX| 15169|.com.|
|         farfesh.com| 142.250.27.27|alt2.aspmx.l.goog...|        MX| 15169|.com.|
|  mammothweather.com|68.178.213.244|mailstore1.secure...|        MX| 26496|.com.|
|        abbccoin.com| 64.233.184.27|alt1.aspmx.l.goog...|        MX| 15169|.com.|
|         brainhi.com| 142.250.27.27|aspmx3.googlemail...|        MX| 15169|.com.|
|        socialbu.com| 142.250.27.27|alt2.aspmx.l.goog...|        MX| 15169|.com.|
|the

In [5]:
# pip install geoip2

In [4]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import socket

import argparse
import geoip2.database

def geo_country(trg_ip):
    with geoip2.database.Reader('GeoLite2-City.mmdb') as geo:
        try:
            res = geo.city(trg_ip)
            country = res.country.name
        
            return country
        except:
            return None

geo_map_udf = udf(geo_country)

final = df.withColumn("location", geo_map_udf(df["ipv4_address"]))

final.show()



+------------------+--------------------+--------------------+----------+------+-----+-------------+
|            domain|        ipv6_address|          query_name|query_type|   asn|  tld|     location|
+------------------+--------------------+--------------------+----------+------+-----+-------------+
|            007.ae|2600:4c00:300:83:...|             007.ae.|        MX| 63410| .ae.|United States|
|            009.am|       2a02:6b8::311|       mx.yandex.ru.|        MX|208722| .am.|       Russia|
|        01math.com|       2a02:6b8::311|      mx.yandex.net.|        MX|208722|.com.|       Russia|
|            021.rs|2a00:1450:4025:40...|ALT2.ASPMX.L.GOOG...|        MX| 15169| .rs.|  Netherlands|
|        027art.com| 240d:c040:1:40::133|    mxdomain.qq.com.|        MX|132203|.com.|    Singapore|
|03ed9035a0801f.com|    2001:430:fff6::4|mx156.hostedmxser...|        MX|  7332|.com.|United States|
|            056.ua|2a00:1450:4025:c0...|alt4.aspmx.l.goog...|        MX| 15169| .ua.|     

In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import socket

import argparse
import geoip2.database
    
def geo_latitude(trg_ip):
    with geoip2.database.Reader('GeoLite2-City.mmdb') as geo:
        try:
            res = geo.city(trg_ip)
            latitude = res.location.latitude

            return latitude
        except:
            return None

geo_lati_udf = udf(geo_latitude)

final = final.withColumn("latitude", geo_lati_udf(final["ipv4_address"]))

final.show()

+------------------+--------------------+--------------------+----------+------+-----+-------------+--------+
|            domain|        ipv6_address|          query_name|query_type|   asn|  tld|     location|latitude|
+------------------+--------------------+--------------------+----------+------+-----+-------------+--------+
|            007.ae|2600:4c00:300:83:...|             007.ae.|        MX| 63410| .ae.|United States|  37.751|
|            009.am|       2a02:6b8::311|       mx.yandex.ru.|        MX|208722| .am.|       Russia| 55.7483|
|        01math.com|       2a02:6b8::311|      mx.yandex.net.|        MX|208722|.com.|       Russia| 55.7483|
|            021.rs|2a00:1450:4025:40...|ALT2.ASPMX.L.GOOG...|        MX| 15169| .rs.|  Netherlands| 52.9597|
|        027art.com| 240d:c040:1:40::133|    mxdomain.qq.com.|        MX|132203|.com.|    Singapore|  1.3673|
|03ed9035a0801f.com|    2001:430:fff6::4|mx156.hostedmxser...|        MX|  7332|.com.|United States|  37.751|
|         

In [6]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import socket

import argparse
import geoip2.database
    
def geo_longitude(trg_ip):
    with geoip2.database.Reader('GeoLite2-City.mmdb') as geo:
        try:
            res = geo.city(trg_ip)
            longitude = res.location.longitude

            return str(longitude)
        except:
            return None

geo_longi_udf = udf(geo_longitude)

final = final.withColumn("longitude", geo_longi_udf(final["ipv4_address"]))

final.show()

+------------------+--------------------+--------------------+----------+------+-----+-------------+--------+---------+
|            domain|        ipv6_address|          query_name|query_type|   asn|  tld|     location|latitude|longitude|
+------------------+--------------------+--------------------+----------+------+-----+-------------+--------+---------+
|            007.ae|2600:4c00:300:83:...|             007.ae.|        MX| 63410| .ae.|United States|  37.751|  -97.822|
|            009.am|       2a02:6b8::311|       mx.yandex.ru.|        MX|208722| .am.|       Russia| 55.7483|  37.6171|
|        01math.com|       2a02:6b8::311|      mx.yandex.net.|        MX|208722|.com.|       Russia| 55.7483|  37.6171|
|            021.rs|2a00:1450:4025:40...|ALT2.ASPMX.L.GOOG...|        MX| 15169| .rs.|  Netherlands| 52.9597|   4.7517|
|        027art.com| 240d:c040:1:40::133|    mxdomain.qq.com.|        MX|132203|.com.|    Singapore|  1.3673| 103.8014|
|03ed9035a0801f.com|    2001:430:fff6::4